In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn import GELU
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import bisect
from myPytorchModels import Seq2SeqTimeSeriesTransformer

In [2]:
# Prepare the Data ---------------------------------------------------------------------

# ------------------------
# Load info
# ------------------------
info_df = pd.read_csv("info.csv")
fs = info_df.iloc[0, 5]                  # sampling frequency (Hz)

# ------------------------
# Load baseline data
# ------------------------
baseline_df = pd.read_csv("baselinedataraw.csv")
baseline_time = baseline_df.iloc[:, 0].values            # time column (seconds)
baseline_data = baseline_df.iloc[:, 1:].values           # data columns

# ------------------------
# Load main data
# ------------------------
df = pd.read_csv("dataraw.csv")
time = df.iloc[:, 0].values            # time column (seconds)
data = df.iloc[:, 1:].values           # data columns

# ------------------------
# Load events
# ------------------------
events_df = pd.read_csv("events.csv")
event_times = events_df.iloc[:, 0].values  # assume first column is event time in seconds
event_times = np.sort(event_times)         # ensure sorted

# For fast lookup using binary search
def count_events_in_window(t, window=0.2):
    """
    Count how many event_times fall in (t - window, t].
    Uses bisect for O(log n) search.
    """
    left = bisect.bisect_right(event_times, t - window)
    right = bisect.bisect_right(event_times, t)
    return right - left

# ------------------------
# Compute event count for each row
# ------------------------
event_counts = np.array([count_events_in_window(t, 1.2/fs) for t in time])
event_counts = event_counts.reshape(-1, 1)

# Append event_counts as an additional input feature
data_aug = np.hstack([data, event_counts])
# Now each input row has: [original data..., event_count]

# ------------------------
# Determine outliers
# ------------------------
threshsd = 3 # standard deviations 
threshprop = .5 # proportion of features
BLmean = np.mean(baseline_data, axis=0)
BLstd = np.std(baseline_data, axis=0)
BLisout = np.abs(baseline_data - BLmean) > (threshsd * BLstd)
BLisnoise = np.sum(BLisout, axis=1) > (threshprop * baseline_data.shape[1])
isout = np.abs(data - BLmean) > (threshsd * BLstd)
isnoise = np.sum(isout, axis=1) > (threshprop * data.shape[1])

# ------------------------
# Build input-output pairs using the _ ms rule
# ------------------------
dt_target = 1/fs      # s
dt_tol = 0.15 * dt_target
seq_len = 32                   # samples
hzn_len = 16                   # samples
drow_target = int(dt_target * fs)  # number of rows 

X_list = []
Y_list = []

# create sliding windows
def create_windows(data, seq_len=128, horizon=1):
    X, Y = [], []
    for i in range(len(data) - seq_len - horizon + 1):
        X.append(data[i:i+seq_len])
        Y.append(data[(i+seq_len):(i+seq_len+horizon)])
    return np.array(X), np.array(Y)

# --- baseline data ---
Nbl = len(baseline_df)
inputs = []
for i in range(Nbl - drow_target):
    dt = baseline_time[i+drow_target] - baseline_time[i]
    if (abs(dt - dt_target) <= dt_tol) and (not BLisnoise[i]):
        inputs.append(baseline_data[i]) 
    else:
        if len(inputs) > seq_len+hzn_len:
            x, y = create_windows(inputs, seq_len, hzn_len)
            if x is not None:
                X_list.append(x)
                Y_list.append(y)
            inputs = []
# catch trailing segment
if len(inputs) > seq_len+hzn_len:
    x, y = create_windows(inputs, seq_len, hzn_len)
    if x is not None:
        X_list.append(x)
        Y_list.append(y)

X = np.concatenate(X_list, axis=0)
Y = np.concatenate(Y_list, axis=0)

X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

print("Pairs created:", len(X))
print("Input shape :", X.shape)   
print("Output shape:", Y.shape)

num_feat = X.shape[-1]

# -----------------------------------------------------------------------------------------------


Pairs created: 1419465
Input shape : torch.Size([1419465, 32, 7])
Output shape: torch.Size([1419465, 16, 7])


In [5]:
# Initialize the Model, Loss Function, and Optimizer

test_size=0.7
batch_size = 32

#model = TimeSeriesTransformer(dim_in=num_feat, dim_model=8, num_heads=2, num_layers=2, dim_ff=16, pos_len=seq_len)
model = Seq2SeqTimeSeriesTransformer(dim_in=num_feat, dim_out=num_feat, seq_len=seq_len, horizon=hzn_len, dim_model=8, num_heads=2, num_encoder_layers=2, num_decoder_layers=2, dim_ff=16)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train / test
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size, random_state=42)
train_N = int((1 - test_size) * len(X))
X_train = X[:train_N]
Y_train = Y[:train_N]
X_test = X[train_N:]
Y_test = Y[train_N:]

# Create TensorDatasets
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
all_loader = DataLoader(TensorDataset(X, Y), shuffle=False)

In [6]:
# data baseline characteristics as reference for loss 
mean_y = Y_train.mean(dim=0)
std_y = Y_train.std(dim=0)
var_y = std_y ** 2
var_per_feat = np.var(Y_train.numpy(), axis=0)  # redundant?

In [11]:
def train_epoch(model, dataloader, optimizer, loss_fn, device, teacher_forcing_ratio=1.0):
    """
    One epoch of training with optional teacher forcing ratio.
    teacher_forcing_ratio=1.0 -> always use ground truth as decoder input (fastest, stable)
    teacher_forcing_ratio=0.0 -> always use model predictions as previous inputs (full AR)
    Usually we use teacher forcing during training; for robustness you can mix.
    """
    model.train()
    total_loss = 0.0
    for xb, yb in dataloader:
        xb = xb.to(device).float()   # (batch, seq_len, dim_in)
        yb = yb.to(device).float()   # (batch, horizon, dim_out)

        b, H, D_out = yb.shape

        # Build tgt_in for teacher forcing:
        # tgt_in shape: (batch, H, dim_out)
        # convention: tgt_in[:, 0, :] is "start token" (zeros)
        start = torch.zeros((b, 1, D_out), device=device, dtype=xb.dtype)
        if teacher_forcing_ratio >= 1.0:
            tgt_in = torch.cat([start, yb[:, :-1, :]], dim=1)
            # forward
            logits = model(xb, tgt_in)   # (batch, H, dim_out)
        else:
            # scheduled sampling: mix ground truth and previous predictions
            # simple implementation: step through horizon building tgt_in progressively
            preds = []
            prev = start
            for t in range(H):
                # build current input sequence
                if t == 0:
                    cur_tgt = prev
                else:
                    cur_tgt = torch.cat([start, torch.cat(preds, dim=1)], dim=1)
                logits_t = model(xb, cur_tgt)  # (batch, t+1, dim_out)
                next_pred = logits_t[:, -1:, :]  # (b,1,d)
                # decide whether to teacher-force for next step
                if torch.rand(1).item() < teacher_forcing_ratio:
                    chosen = yb[:, t:t+1, :]
                else:
                    chosen = next_pred.detach()
                preds.append(chosen)
            logits = torch.cat(preds, dim=1)

        loss = loss_fn(logits, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.shape[0]
    return total_loss / len(dataloader.dataset)


def validate_single_step(model, test_loader, device, criterion):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)        # [B, seq_len, dim_in]
            y = y.to(device)        # [B, horizon, dim_out]

            """
            _, pred = model(x, target_seq_len=1)  # pred: [B, 1, dim_out]
            pred = pred[:, -1, :]                  # → [B, dim_out]
            """
            pred = model.generate(x, horizon=y.size(1))  # [B, horizon, dim_out]

            loss = criterion(pred, y)
            total_loss += loss.item() * x.size(0)

    return total_loss / len(test_loader.dataset)



In [12]:
epoch_val_loss = validate_single_step(model, test_loader, torch.device('cpu'), criterion)

KeyboardInterrupt: 

In [13]:
# Step 4: Train the Model
train_size = len(train_loader.dataset)
steps_per_epoch = math.ceil(train_size / batch_size)
print("Train samples:", train_size)
print("Batch size:", batch_size)
print("Batches/epoch:", steps_per_epoch)

model.train()
num_epochs = 100 # max
patience = 10
best_val = float('inf')
no_improve = 0
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # --- train ---
    epoch_train_loss = train_epoch(model, train_loader, optimizer, criterion, torch.device('cpu'), teacher_forcing_ratio=1.0)
    train_losses.append(epoch_train_loss)

    # --- validate ---
    epoch_val_loss = validate_single_step(model, test_loader, torch.device('cpu'), criterion)
    val_losses.append(epoch_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} — train_loss: {epoch_train_loss:.6f}, val_loss: {epoch_val_loss:.6f}")

    # --- Early stopping ---
    if epoch_val_loss < best_val - 1e-9:
        best_val = epoch_val_loss
        no_improve = 0
        # Optionally save best model:
        # torch.save(model.state_dict(), "neural_network_pytorch.pth")
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"No improvement for {patience} epochs — stopping early at epoch {epoch+1}.")
            break

# After loop: plot train/val loss to inspect convergence
plt.plot(train_losses, label='train_loss')
plt.plot(val_losses, label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.show()

Train samples: 425839
Batch size: 32
Batches/epoch: 13308


KeyboardInterrupt: 

In [ ]:
# load best model
# model.load_state_dict(torch.load("neural_network_pytorch.pth"))

In [ ]:
# Step 5: Evaluate the Model on Test Data

Y_pred = []
Y_test1 = []

model.eval()
with torch.no_grad():
    total_loss = 0
    for x_batch, y_batch in test_loader:
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        total_loss += loss.item() * x_batch.size(0)  # sum up batch loss
        if y_pred.shape[0] == seq_len:
            Y_pred.append(y_pred)
            Y_test1.append(y_batch)

    avg_loss = total_loss / len(test_dataset)
    print(f"Test Loss: {avg_loss:.4f}")

Y_pred_np = np.array(Y_pred)
Y_pred_np = Y_pred_np.reshape(-1, num_feat)
Y_test_np = Y_test.numpy()
Y_test1_np = np.array(Y_test1)
Y_test1_np = Y_test1_np.reshape(-1, num_feat)

Y_null_all_np = X.numpy()[:, -1, :Y.shape[1]]
Y_null_test_np = X_test.numpy()[:, -1, :Y.shape[1]]

MSE_per_feat = np.mean((Y_test1_np - Y_pred_np) ** 2, axis=0)
MSE_per_feat_null = np.mean((Y_test_np - Y_null_test_np) ** 2, axis=0)
feats = np.arange(1, Y.shape[1]+1)
barwid = .35

plt.figure(figsize=(15,5))
plt.bar(feats - barwid, var_per_feat, width=barwid, label='Output Variance')
plt.bar(feats, MSE_per_feat_null, width=barwid, label='Null MSE')
plt.bar(feats + barwid, MSE_per_feat, width=barwid, label='Test MSE')
plt.xlabel('Output Feature')
plt.ylabel('Value')
plt.title('Output Feature Variance vs Test MSE')
plt.legend()
plt.show()

In [ ]:
X_all_np = X.numpy()
Y_all_np = Y.numpy()

Y_all_pred = []

model.eval()
with torch.no_grad():
    total_loss = 0
    for x_batch, y_batch in all_loader:
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        total_loss += loss.item() * x_batch.size(0)  # sum up batch loss
        Y_all_pred.append(y_pred)

    avg_loss = total_loss / len(test_dataset)
    print(f"Test+Train Loss: {avg_loss:.4f}")

Y_pred_all_np = np.array(Y_all_pred)
Y_pred_all_np = Y_pred_all_np.reshape(-1, num_feat)

In [ ]:
# simulations 
simdur = int(0.2 * fs) # samples 
plotdomain = 1000 * np.array([-1, 1]) + train_N

Ysim = []
i0 = plotdomain[0]
model.eval()
while i0+simdur < plotdomain[1]:
    xi = torch.tensor(X_all_np[i0, :, :].reshape(1,-1,num_feat), dtype=torch.float32)
    for i in range(simdur):
        with torch.no_grad():
            yi = model(xi).numpy().flatten()
        Ysim.append(yi)
        # prepare next input
        if i < simdur - 1:
            #event_count_next = X_all_np[i0 + i + 1, -1]  # keep using original event count
            #xi = torch.tensor(np.hstack([yi, event_count_next]).reshape(1, -1), dtype=torch.float32)
            xi = torch.tensor(np.vstack([xi[0, 1:, :], yi]).reshape(1,-1,X.shape[-1]), dtype=torch.float32)
    i0 += simdur
    print("Simulating:", (i0-plotdomain[0])/(plotdomain[1]-plotdomain[0]), " complete." )

Ysim = np.array(Ysim)
plotxval = np.arange(len(Ysim)) + plotdomain[0]

In [ ]:
# show several examples 

iMSE = np.argsort(MSE_per_feat)
iVAR = np.argsort(var_per_feat)
iLRN = np.argsort(MSE_per_feat / var_per_feat)
iToPlot = [iMSE[:2], iMSE[-2:], iVAR[:2], iVAR[-2:], iLRN[:2], iLRN[-2:]]
iToPlot = list(set([i for sublist in iToPlot for i in sublist]))

plt.figure(figsize=(15,20))
iPlot = 1
for i in iToPlot:
    plt.subplot(len(iToPlot), 1, iPlot)
    plt.plot(Y_all_np[:, i], label='True')
    plt.plot(Y_pred_all_np[:, i], label='Predicted', linestyle='--')
    plt.plot(Y_null_all_np[:, i], label='Null', linestyle=':')
    plt.plot(plotxval, Ysim[:,i], label='Simulated', linestyle='-.')
    plt.xlim(plotdomain)

    # set the y limits to be slightly larger than the min/max of true values in the plotdomain
    y_min = np.min(Y_all_np[plotdomain[0]:plotdomain[1], i])
    y_max = np.max(Y_all_np[plotdomain[0]:plotdomain[1], i])
    y_range = y_max - y_min
    plt.ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
    
    plt.title(f'Feature {i+1} - MSE: {MSE_per_feat[i]:.4f}, VAR: {var_per_feat[i]:.4f}')
    plt.legend(loc='upper right')
    iPlot += 1
plt.tight_layout()
plt.show()